In [7]:
import networkx as nx
import matplotlib.pyplot as plt
import os
from datetime import datetime
import imageio
from PIL import Image

def mis(G, output_folder, start_number, full_output=True):
    step_counter = [start_number - 1]
    image_files = []
    total_steps = 0
    line10_calls = 0

    def save_visualize_step(G, title, description, highlight_nodes=None):
        nonlocal total_steps
        step_counter[0] += 1
        total_steps += 1
        if not full_output:
            return
        
        pos = nx.spring_layout(G)
        plt.figure(figsize=(12, 9))
        nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500)
        if highlight_nodes:
            nx.draw_networkx_nodes(G, pos, nodelist=highlight_nodes, node_color='red', node_size=600)
        plt.title(f"Step {step_counter[0]}: {title}\n{description}\n"
                  f"Nodes: {len(G)}, Edges: {G.number_of_edges()}", wrap=True)
        filename = f"{output_folder}/{step_counter[0]:03d}_graph_{len(G)}nodes_{G.number_of_edges()}edges_MIS{len(highlight_nodes) if highlight_nodes else 'unknown'}.png"
        plt.savefig(filename, dpi=300, bbox_inches='tight')
        plt.close()
        
        with Image.open(filename) as img:
            img = img.resize((1200, 900), Image.LANCZOS)
            img.save(filename)
        
        image_files.append(filename)
        print(f"Saved: {filename}")

    if len(G) == 0:
        save_visualize_step(G, "Empty Graph", "The graph is empty. MIS size is 0.")
        return 0, image_files, total_steps, line10_calls

    save_visualize_step(G, "Initial Graph", "Starting the MIS algorithm with the original graph.")

    max_degree = max(dict(G.degree()).values())
    if max_degree <= 2:
        mis_set = nx.maximal_independent_set(G)
        #note: the maximal independent set of graphs with maximum degree 2 is the same
        # as its maximum independent set
        save_visualize_step(G, "MIS for Graph with Max Degree <= 2", 
                            f"Found MIS of size {len(mis_set)}. When the maximum degree is 2 or less, we can efficiently compute the MIS.",
                            mis_set)
        return len(mis_set), image_files, total_steps, line10_calls

    degree_one_nodes = [n for n, d in G.degree() if d == 1]
    if degree_one_nodes:
        v = degree_one_nodes[0]
        G_minus_N_v = G.copy()
        G_minus_N_v.remove_nodes_from(list(G.neighbors(v)) + [v])
        save_visualize_step(G_minus_N_v, f"Removed Node {v} (Degree 1) and Its Neighbors", 
                            f"When we find a node of degree 1, we include it in the MIS and remove its neighbor. This reduces the problem size.")
        sub_mis_size, sub_image_files, sub_steps, sub_line10 = mis(G_minus_N_v, output_folder, step_counter[0] + 1, full_output)
        image_files.extend(sub_image_files)
        total_steps += sub_steps
        #line10_calls += sub_line10
        return 1 + sub_mis_size, image_files, total_steps, line10_calls

    if not nx.is_connected(G):
        components = list(nx.connected_components(G))
        G1 = G.subgraph(components[0])
        G_minus_G1 = G.copy()
        G_minus_G1.remove_nodes_from(components[0])
        save_visualize_step(G1, "Connected Component G1", 
                            "The graph is not connected. We solve MIS for each connected component separately.")
        save_visualize_step(G_minus_G1, "Remaining Graph after Removing G1", 
                            "We'll solve MIS for this part separately and combine the results.")
        sub_mis_size1, sub_image_files1, sub_steps1, sub_line10_1 = mis(G1, output_folder, step_counter[0] + 1, full_output)
        sub_mis_size2, sub_image_files2, sub_steps2, sub_line10_2 = mis(G_minus_G1, output_folder, step_counter[0] + len(sub_image_files1) + 1, full_output)
        image_files.extend(sub_image_files1)
        image_files.extend(sub_image_files2)
        total_steps += sub_steps1 + sub_steps2
        #line10_calls += sub_line10_1 + sub_line10_2
        return sub_mis_size1 + sub_mis_size2, image_files, total_steps, line10_calls

    max_degree_node = max(G.degree(), key=lambda x: x[1])[0]
    G_minus_N_v = G.copy()
    G_minus_N_v.remove_nodes_from(list(G.neighbors(max_degree_node)) + [max_degree_node])
    G_minus_v = G.copy()
    G_minus_v.remove_node(max_degree_node)
    
    save_visualize_step(G_minus_N_v, f"Removed Node {max_degree_node} (Max Degree) and Its Neighbors", 
                        f"We're exploring the case where node {max_degree_node} is in the MIS. We remove it and its neighbors from the graph.")
    save_visualize_step(G_minus_v, f"Removed Only Node {max_degree_node}", 
                        f"We're also exploring the case where node {max_degree_node} is not in the MIS. We only remove it from the graph.")
    
    line10_calls += 1
    sub_mis_size1, sub_image_files1, sub_steps1, sub_line10_1 = mis(G_minus_N_v, output_folder, step_counter[0] + 1, full_output)
    sub_mis_size2, sub_image_files2, sub_steps2, sub_line10_2 = mis(G_minus_v, output_folder, step_counter[0] + len(sub_image_files1) + 1, full_output)
    image_files.extend(sub_image_files1)
    image_files.extend(sub_image_files2)
    total_steps += sub_steps1 + sub_steps2
    line10_calls += sub_line10_1 + sub_line10_2
    return max(1 + sub_mis_size1, sub_mis_size2), image_files, total_steps, line10_calls

def create_video(image_files, output_folder, fps=1):
    video_path = os.path.join(output_folder, "mis_algorithm_progress.mp4")
    with imageio.get_writer(video_path, fps=fps) as writer:
        for filename in sorted(image_files):
            image = imageio.imread(filename)
            writer.append_data(image)
    print(f"Video saved: {video_path}")

def run_mis_algorithm(G, solution_name, start_number, full_output=True):
    main_folder = "MIS_algo_graph"
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    solution_folder = f"{solution_name}_{timestamp}"
    output_folder = os.path.join(main_folder, solution_folder)
    os.makedirs(output_folder, exist_ok=True)

    # Find MIS
    mis_size, image_files, total_steps, line10_calls = mis(G, output_folder, start_number, full_output)

    print(f"Size of Maximum Independent Set: {mis_size}")
    print(f"Total number of steps: {total_steps}")
    print(f"Number of times line 10 was called: {line10_calls}")

    # Visualize final graph with MIS
    mis_nodes = nx.maximal_independent_set(G)
    pos = nx.spring_layout(G)
    plt.figure(figsize=(12, 9))
    nx.draw(G, pos, with_labels=True, node_color='lightblue', node_size=500)
    nx.draw_networkx_nodes(G, pos, nodelist=mis_nodes, node_color='red', node_size=600)
    plt.title(f"Final Graph with Maximum Independent Set\nMIS size: {len(mis_nodes)}\n"
              f"Nodes: {len(G)}, Edges: {G.number_of_edges()}\n"
              f"Total steps: {total_steps}, Line 10 calls: {line10_calls}\n"
              f"Red nodes represent the Maximum Independent Set. These nodes are not connected to each other.",
              wrap=True)
    final_filename = os.path.join(output_folder, f"final_graph_MIS{len(mis_nodes)}_steps{total_steps}.png")
    plt.savefig(final_filename, dpi=300, bbox_inches='tight')
    plt.close()
    
    with Image.open(final_filename) as img:
        img = img.resize((1200, 900), Image.LANCZOS)
        img.save(final_filename)
    
    print(f"Saved final graph: {final_filename}")
    
    if full_output:
        image_files.append(final_filename)
        create_video(image_files, output_folder)

    return mis_size, output_folder, total_steps, line10_calls

In [14]:
# Example usage
# random generate graph with probability
G1 = nx.gnp_random_graph(10, 0.3)

In [15]:
print("\nRunning with minimal output:")
mis_size2, folder2, total_steps2, line10_calls2 = run_mis_algorithm(G1, "random_graph_10_15_minimal", start_number=1, full_output=False)
print(f"Minimal output solution saved in: {folder2}")
print(f"Total steps: {total_steps2}")
print(f"Line 10 calls: {line10_calls2}")


Running with minimal output:
Size of Maximum Independent Set: 5
Total number of steps: 12
Number of times line 10 was called: 0
Saved final graph: MIS_algo_graph/random_graph_10_15_minimal_20240729_162434/final_graph_MIS4_steps12.png
Minimal output solution saved in: MIS_algo_graph/random_graph_10_15_minimal_20240729_162434
Total steps: 12
Line 10 calls: 0


In [10]:

# print("Running with full output:")
# mis_size1, folder1, total_steps1, line10_calls1 = run_mis_algorithm(G1, "random_graph_10_15_full", start_number=1, full_output=True)
# print(f"Full output solution saved in: {folder1}")
# print(f"Total steps: {total_steps1}")
# print(f"Line 10 calls: {line10_calls1}")

Running with full output:
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/001_graph_10nodes_12edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/002_graph_8nodes_11edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/003_graph_8nodes_11edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/004_graph_6nodes_7edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/005_graph_6nodes_7edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/006_graph_1nodes_0edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/007_graph_5nodes_7edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/008_graph_1nodes_0edges_MISunknown.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/009_graph_1nodes_0edges_MIS1.png
Saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/010_g

/var/folders/p2/tltdzcl142g2b8gt5mll07w00000gn/T/ipykernel_13041/29961957.py:109: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1200, 900) to (1200, 912) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
/var/folders/p2/tltdzcl142g2b8gt5mll07w00000gn/T/ipykernel_13041/29961957.py:109: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2

Video saved: MIS_algo_graph/random_graph_10_15_full_20240729_160149/mis_algorithm_progress.mp4
Full output solution saved in: MIS_algo_graph/random_graph_10_15_full_20240729_160149
Total steps: 15
Line 10 calls: 0


In [37]:
nx.maximal_independent_set(G1)

[1, 4, 7, 6]

In [38]:
# make g1 in to adjacency matrix
G1_adj = nx.adjacency_matrix(G1).todense()

ModuleNotFoundError: No module named 'scipy'